In [26]:
import pandas as pd
import datarobot as dr
from datetime import datetime
from datarobot.enums import SERVICE_STAT_METRIC
from datarobot.helpers.partitioning_methods import construct_duration_string
import os
import time

In [2]:
# 
# Connect to DataRobot  
#
# Replace the API token and username with yours, along with the 
# depolyment id and/or project and model ids you are using.  
#

USERNAME = os.environ['DATAROBOT_USERNAME']
API_TOKEN = os.environ['DATAROBOT_API_TOKEN']
ENDPOINT = 'https://app.datarobot.com/api/v2'

dr.Client(token=API_TOKEN, endpoint=ENDPOINT)  

## Make predictions and get predictions explanations

### Pixability - Deployments

In [32]:
pred_file = './data/new_pix_testing_first10rows.csv'
df_pred_data = pd.read_csv(pred_file)

deployment_id = '61dcdd632786724e47573764'
deployment = dr.Deployment.get(deployment_id)
# deployment.

In [19]:
# preds_df = deployment.predict(pred_data)  # No longer works

### Batch prediction

In [23]:
dr.BatchPredictionJob.score_to_file?

In [34]:
#Scoring without Prediction Explanations

t0 = time.time()
job, df_output_preds = dr.BatchPredictionJob.score_pandas(deployment_id, df_pred_data)
print('- score_pandas Done. Total time: %.3f minutes' % ((time.time() - t0)/60))
print()
print(df_pred_data.head())
print()

# dr.BatchPredictionJob.score_to_file(
#     deployment_id,
#     intake_path=pred_file, #Path or Pandas or file-like object
#     output_path='output_predictions.csv'
# )

t0 = time.time()
job = dr.BatchPredictionJob.score(
    deployment_id,
    intake_settings={
        'type': 'localFile',
        'file': pred_file #Path or Pandas or file-like object
    },
    output_settings={
        'type': 'localFile',
        'path': './preds/pixability_output_preds.csv'
    }
)
job.wait_for_completion()
with open("./preds/pixability_output_preds.csv", "wb") as f:
    job.download(f)
print('- score Done. Total time: %.3f minutes' % ((time.time() - t0)/60))
print()

# #Scoring With Prediction Explanations - not available for multiclas
# t0 = time.time()
# job = dr.BatchPredictionJob.score(
#     deployment_id,
#     intake_settings={
#         'type': 'localFile',
#         'file': pred_file #Path or Pandas or file-like object
#     },
#     output_settings={
#         'type': 'localFile',
#         'path': './preds/pixability_output_preds_pred_expl.csv'
#     },
#     max_explanations=3 #Compute prediction explanations for this amount of features
# )
# job.wait_for_completion()
# with open("./pixability_output_preds_pred_expl.csv", "wb") as f:
#     job.download(f)
# print('- score with pred expl Done. Total time: %.3f minutes' % ((time.time() - t0)/60))
# print()

Streaming DataFrame as CSV data to DataRobot
Created Batch Prediction job ID 61de6db234d2451cc54a2f92
Waiting for DataRobot to start processing
Job has started processing at DataRobot. Streaming results.
- score_pandas Done. Total time: 0.800 minutes

                                                text       label
0  Người Dân Củ Chi Bỏ Hết Công Việc Đồng Án Để T...  Brand Safe
1  🚂 POCOYO E NINA - Viagem no tempo  [114 minuto...  Brand Safe
2  Tubarão Bebê, Cinco Patinhos e Mais Músicas In...  Brand Safe
3  BTS (방탄소년단) - DNA [Music Bank COMEBACK / 2017....  Brand Safe
4  Alan Walker ft  Sia   Diamond Heart Lyrics Vid...  Brand Safe

- score Done. Total time: 1.322 minutes


ClientError: 422 client error: {'message': 'Invalid field data', 'errors': {'maxExplanations': 'Prediction Explanations are not initialized for the model'}}

### Reqest.post

In [ ]:
headers = {
    # As default, we expect CSV as input data.
    # Should you wish to supply JSON instead,
    # comment out the line below and use the line after that instead:
    'Content-Type': 'text/plain; charset=UTF-8',
    # 'Content-Type': 'application/json; charset=UTF-8',

    'Authorization': 'Bearer {}'.format(API_KEY),
    'DataRobot-Key': DATAROBOT_KEY,
}

url = API_URL.format(deployment_id=deployment_id)

# Prediction Explanations:
# See the documentation for more information:
# https://app.datarobot.com/docs/predictions/api/dr-predapi.html#request-pred-explanations
# Should you wish to include Prediction Explanations or Prediction Warnings in the result,
# Change the parameters below accordingly, and remove the comment from the params field below:

params = {
    # If explanations are required, uncomment the line below
    # 'maxExplanations': 3,
    # 'thresholdHigh': 0.5,
    # 'thresholdLow': 0.15,
    # Uncomment this for Prediction Warnings, if enabled for your deployment.
    # 'predictionWarningEnabled': 'true',
}
# Make API request for predictions
predictions_response = requests.post(
    url,
    data=data,
    headers=headers,
    # Prediction Explanations:
    # Uncomment this to include explanations in your prediction
    # params=params,
)
_raise_dataroboterror_for_status(predictions_response)
# Return a Python dict following the schema in the documentation
predictions_response.json()